# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pp

# Import API key
 = "AIzaSyAuuZONjAi_0_gJw6P3vOY-bBknHQIcID5F8w"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
file = "data_frame_export.csv"
weather_df = pd.read_csv(file)
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Marsh Harbour,5,BS,1584234003,81,26.54,-77.06,73.96,10.56
1,Soligalich,100,RU,1584234199,72,59.08,42.29,21.40,5.35
2,Punta Arenas,100,CL,1584234199,76,-53.15,-70.92,57.00,25.28
3,Albany,90,US,1584233942,55,42.60,-73.97,39.99,9.17
4,Puerto Ayora,20,EC,1584234199,74,-0.74,-90.35,82.40,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [174]:
# Configure gmaps
gmaps.configure(api_key) 

 # Store latitude and longitude in locations
locations = weather_df[["Lat", "Lon"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
#Specify plotting parameters
figure_layout = {
    'width': '1000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px'
}

#Plot heatmap
gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [182]:
#Narrow down the DataFrame to find your ideal weather condition. For example:

ideal_conditions_df = weather_df.loc[(weather_df["Max Temp"] > 70) & 
                                     (weather_df["Max Temp"] < 80) & 
                                     (weather_df["Wind Speed"] < 10) & 
                                     (weather_df["Cloudiness"] == 0)]
#ceftamin_only_df = joined_data.loc[(joined_data["Drug Regimen"] == "Ceftamin")]

#lat = ideal_conditions_df.loc[(ideal_conditions_df["Lat"])]

#Connect Lat & Lon
#ideal_conditions_df['Lat & Lon'] = ideal_conditions_df.loc(ideal_conditions_df["Lat"].map(str)) + "," + ideal_conditions_df.loc(ideal_conditions_df["Lon"].map(str))
#ideal_conditions_df.head()

#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

hotel_name = []
hotel_city = []
hotel_country = []
hotel_lat = []
hotel_lng = []

for index, row in ideal_conditions_df.iterrows():

    coords = f"{row['Lat']},{row['Lon']}"

    # geocoordinates
    target_coordinates = coords
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": api_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
    if response['status'] != "ZERO_RESULTS":
    
        hotel_name.append(response["results"][0]["name"])
        hotel_country.append(response["results"][0]["plus_code"]["compound_code"])
        hotel_city.append(response["results"][0]["vicinity"])
        hotel_lat.append(response["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(response["results"][0]["geometry"]["location"]["lng"])
    
    else:
        print(f"No hotels within 5000m! of {coords}")

pp.pprint (response)
hotels_df = pd.DataFrame({"Name": hotel_name,
                          "City": hotel_city,
                          "Country": hotel_country,
                          "Lat": hotel_lat,
                          "Lon": hotel_lng})
hotels_df

No hotels within 5000m! of 22.68,-3.98
No hotels within 5000m! of 16.41,-3.67
{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': 13.5887132, 'lng': -89.8342137},
                           'viewport': {'northeast': {'lat': 13.5900961302915,
                                                      'lng': -89.8329496697085},
                                        'southwest': {'lat': 13.5873981697085,
                                                      'lng': -89.83564763029152}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
              'id': '18cd8cefd499577d03f9f78fa6b29906af6ebc3e',
              'name': 'Villavela Hotel Boutique',
              'photos': [{'height': 3120,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/118297599492986181777">Manuel '
                                                'Baltazar Cruz</a>'],

,Name,City,Country,Lat,Lon
0,Casa Kiin,Carretera Valladolid - Felipe Carrillo Puerto ...,"JQXP+2G Chichimila, Yucatan, Mexico",20.647601,-88.213687
1,Pondok Pesantren Al Muhajirin Atambua,Beirafu,"VVRR+43 Atambua, Berdao, Belu Regency, East Nu...",-9.109632,124.890197
2,Juvinal Martin,Atsabe,"399W+QP Atsabe, Timor-Leste",-8.930576,125.396797
3,Villavela Hotel Boutique,"Avenida Miramar, Acajutla","H5Q8+F8 Acajutla, El Salvador",13.588713,-89.834214


In [183]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
    
hotel_coords = hotels_df[["Lat", "Lon"]].astype(float)

info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]


hotel_layer = gmaps.symbol_layer(
    hotel_coords, info_box_content = hotel_info, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2)

fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map